In [10]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

In [11]:
file_path = os.getcwd()

fig_path = file_path +"\\figures"

In [12]:
print("")
print("**********************************************************************************")
print("Downloading and processing BLS file")
print("")

url = "https://data.bls.gov/cew/data/files/2017/csv/2017_annual_singlefile.zip"
# This will read in the annual, single file. It's big, but has all we want...

r = requests.get(url) 

# convert bytes to zip file  
bls_sf = zf.ZipFile(io.BytesIO(r.content)) 
print('Type of zipfile object:', type(bls_sf))

clist = ['area_fips', 'own_code', 'industry_code', 'agglvl_code', 'size_code',
       'year', 'disclosure_code', 'annual_avg_estabs',
       'annual_avg_emplvl', 'total_annual_wages','avg_annual_pay']

df = pd.read_csv(bls_sf.open(bls_sf.namelist()[0]), usecols= clist)

# SHOULD PRESPECIFY TYPES TO 

df.head()


**********************************************************************************

Type of zipfile object: <class 'zipfile.ZipFile'>


/var/folders/60/4kc0p2cn3w5_067bvcmybpbw0000gn/T/ipykernel_50089/1077924030.py:19: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(bls_sf.open(bls_sf.namelist()[0]), usecols= clist)


,area_fips,own_code,industry_code,agglvl_code,size_code,year,disclosure_code,annual_avg_estabs,annual_avg_emplvl,total_annual_wages,avg_annual_pay
0,1000,0,10,50,0,2017,NaN,124881,1936819,89088710816,45997
1,1000,1,10,51,0,2017,NaN,1208,53131,4339038631,81668
2,1000,1,102,52,0,2017,NaN,1208,53131,4339038631,81668
3,1000,1,1021,53,0,2017,NaN,610,11173,716001109,64083
4,1000,1,1022,53,0,2017,NaN,2,12,369309,30354


In [13]:
df.shape
# df.to_csv("/Users/changjay/Desktop/Pandas-to-STATA Project/countylevel_tariffs_and_exports/step1_initial.csv")
# df.to_stata("/Users/changjay/Desktop/Pandas-to-STATA Project/countylevel_tariffs_and_exports/countylevel_tariffs_and_exports.dta")

(3561646, 11)

In [14]:
'''
location = "/Users/changjay/Desktop/Pandas-to-STATA Project/countylevel_tariffs_and_exports/initial.csv"

df = pd.read_csv(location)
df.head()
'''

'\nlocation = "/Users/changjay/Desktop/Pandas-to-STATA Project/countylevel_tariffs_and_exports/initial.csv"\n\ndf = pd.read_csv(location)\ndf.head()\n'

In [15]:
NAICS_county_level = 75 
# This is the code that will select only counties at the 3 digit NAICS level

df_county = df[df.agglvl_code == NAICS_county_level].copy()
df_county.shape

(274608, 11)

In [16]:
df_county = df_county[df_county.own_code == 5]
# Only grab private stuff
df_county.shape
# df_county.head()

(203667, 11)

In [17]:
df_county['area_fips'] = df['area_fips'].astype(str) # the author forgot this command.
### The author forgot to transform the data type of area_fips, but if this command is run with other commands, there will be no error warning.
### So I add this command myself. However, the result is just slightly different.
# df_county.area_fips.dtype
df_county = df_county[(df_county.area_fips.str[0:2] != "72") & (df_county.area_fips.str[0:2] != "78")
              & (df_county.area_fips.str[0:2] != "02") & (df_county.area_fips.str[0:2] != "15")]
# Drop puerto rico, alaska, hawaii...this mayb not be doing what I think it is...as it looks like these guys are there
# Does not matter as analysis is performed withthem, drop them when do the map. 
df_county.shape

(199393, 11)

In [18]:
df_county["sup_ind"] = df_county.industry_code.str[1].astype(int)
# sometimes there are super industries floating around we want to drop them.
# not clear if this matters with the conditioning all ready

df_county = df_county[df_county["sup_ind"] > 0]

df_county.area_fips = df_county.area_fips.astype(str)

df_national = df_county.groupby("industry_code").agg({"annual_avg_emplvl": "sum"})
df_national.shape
df_national.head()

,annual_avg_emplvl
industry_code,
111,527459
112,226092
113,45166
114,5005
115,352782


In [19]:
df_national.reset_index(inplace = True)
df_national.rename({"annual_avg_emplvl":"nat_emplvl"}, axis = 1, inplace = True)
df_national.head()

df_national.to_stata("/Users/changjay/Desktop/Pandas-to-STATA Project/countylevel_tariffs_and_exports/step1.dta") # for checking

,industry_code,nat_emplvl
0,111,527459
1,112,226092
2,113,45166
3,114,5005
4,115,352782


In [20]:
df_county.annual_avg_emplvl.sum()
# df_national.shape

114942685

In [21]:
print("")
print("**********************************************************************************")
print("Downloading and processing Trade Data")
print("")

my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"
# This is my key. I'm nice and I have it posted. If you will be doing more with this
# please get your own key!

end_use = "hs?get=E_COMMODITY,CTY_CODE,ALL_VAL_MO,CTY_NAME"

url = "https://api.census.gov/data/timeseries/intltrade/exports/" + end_use 
url = url + my_key + "&time==from+2015-01" + "&COMM_LVL=HS6"

url = url + "&CTY_CODE=5700"

r = requests.get(url) 

df_china_trade = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
# Note that the first entry is the labels

df_china_trade.columns = r.json()[0]

df_china_trade.time = pd.to_datetime(df_china_trade.time, format="%Y-%m")
# This is so I can call this correctly...

df_china_trade["china_trade"] = df_china_trade.ALL_VAL_MO.astype(float)

df_china_trade.E_COMMODITY = df_china_trade.E_COMMODITY.astype(str)

df_china_trade.tail(10)


**********************************************************************************



AttributeError: 'DataFrame' object has no attribute 'dtype'

In [29]:
df_china_trade.shape

(348612, 8)

In [24]:
df_china_trade.to_csv("/Users/changjay/Desktop/Pandas-to-STATA Project/countylevel_tariffs_and_exports/step2_china.csv")

In [26]:
r

<Response [200]>

In [27]:
end_use = "hs?get=E_COMMODITY,ALL_VAL_MO"

url = "https://api.census.gov/data/timeseries/intltrade/exports/"
url = url + end_use + my_key + "&time==from+2015-01" + "&COMM_LVL=HS6"

r = requests.get(url) 

df_all_trade = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
# Note that the first entry is the labels

df_all_trade.columns = r.json()[0]

df_all_trade.time = pd.to_datetime(df_all_trade.time, format="%Y-%m")
# This is so I can call this correctly...

df_all_trade["total_trade"] = df_all_trade.ALL_VAL_MO.astype(float)

df_all_trade.E_COMMODITY = df_all_trade.E_COMMODITY.astype(str)

df_all_trade.head(10)

,E_COMMODITY,ALL_VAL_MO,time,COMM_LVL,total_trade
0,600544,4546,2015-01-01,HS6,4546.0
1,600542,53239,2015-01-01,HS6,53239.0
2,600541,8175,2015-01-01,HS6,8175.0
3,600534,497405,2015-01-01,HS6,497405.0
4,600533,1993074,2015-01-01,HS6,1993074.0
5,600532,10793848,2015-01-01,HS6,10793848.0
6,600531,1621489,2015-01-01,HS6,1621489.0
7,600523,33574,2015-01-01,HS6,33574.0
8,600522,1305273,2015-01-01,HS6,1305273.0
9,600521,146116,2015-01-01,HS6,146116.0


In [30]:
df_all_trade.to_csv("/Users/changjay/Desktop/Pandas-to-STATA Project/countylevel_tariffs_and_exports/step2_world.csv")

In [6]:
dftrade = df_all_trade.merge(df_china_trade[["E_COMMODITY", "time","china_trade"]], left_on = ["E_COMMODITY", "time"], 
                             right_on = ["E_COMMODITY", "time"], how = "left")

dftrade.set_index("time", inplace = True)

dftrade.drop(["ALL_VAL_MO"], axis = 1, inplace = True)

In [31]:
dftrade.shape

(487163, 4)

In [8]:
dftrade.to_stata("/Users/changjay/Desktop/Pandas-to-STATA Project/countylevel_tariffs_and_exports/step2.dta") # # for checking